# Compute Nearest HYSETS Station

For each ungauged basin, find the closest HYSETS monitoring location.

In [1]:
import os
from time import time
import pandas as pd
import numpy as np
import itertools
import multiprocessing as mp
import random
import geopandas as gpd
from sqlalchemy import create_engine

import psycopg2


In [2]:

conn_params = {
    'dbname': 'basins',
    'user': 'postgres',
    'password': 'pgpass',
    'host': 'localhost',
    'port': '5432',
    'dbname': 'basins',
}
schema_name = 'basins_schema'
table_name = 'hysets_basins'
conn_str = f"postgresql://{conn_params['user']}:{conn_params['password']}@{conn_params['host']}:{conn_params['port']}/{conn_params['dbname']}"

engine = create_engine(conn_str)


In [5]:
max_distance_query = """
SELECT A.id AS A_id,
       B.id AS B_id,
       ST_Distance(A.centroid, B.centroid) AS distance
FROM basins_schema.basin_attributes A
CROSS JOIN LATERAL (
    SELECT id, geom
    FROM basins_schema.hysets_basins
    ORDER BY A.geom <-> B.geom
    LIMIT 1
) B;
"""
region_codes = ['08A', '08B', '08C', '08D',
        '08E', '08F', '08G', '10E',
        'CLR', 'ERK', 'FRA', 'HAY',
        'HGW', 'LRD', 'PCR', 'VCI', 
        'WWA', 'YKR']
for rc in region_codes[2:]:
    indexed_nearest_query = f"""
    SELECT
        A.id AS A_id,
        A.drainage_area_km2,
        B.official_id,
        B.id AS nearest_B_id,
        B.centroid,
        ST_Distance(A.centroid, B.centroid) AS nearest_distance
    FROM 
        basins_schema.basin_attributes A
    CROSS JOIN LATERAL
        (SELECT
             id,
             official_id,
             centroid
         FROM
             basins_schema.hysets_basins
         ORDER BY
             A.centroid <-> centroid
         LIMIT 1) B
    WHERE A.region_code = '{rc}'
    """
    ta = time()
    stn_dists = gpd.read_postgis(indexed_nearest_query, engine, geom_col='centroid')
    stn_dists = stn_dists[['official_id', 'drainage_area_km2', 'nearest_distance']]
    stn_dists['nearest_distance'] /= 1000
    stn_dists['nearest_stn_dist_km'] = stn_dists['nearest_distance'].astype(int)
    stn_dists['drainage_area_km2'] = stn_dists['drainage_area_km2'].astype(int)
    stn_dists = stn_dists[['official_id', 'drainage_area_km2', 'nearest_stn_dist_km']]
    stn_dists.to_csv(f'station_dists/{rc}_dists.csv')
    tb = time()
    t_tot = (tb - ta) / 60
    if t_tot < 1:
        print(f'Query returned {len(stn_dists)} distances in {tb-ta:.2f} seconds for {rc}')
    else:
        print(f'Query returned {len(stn_dists)} distances in {t_tot:.2f} minutes for {rc}')

Query returned 38426 distances in 17.56 seconds for 08C
Query returned 20347 distances in 15.82 seconds for 08D
Query returned 54756 distances in 18.58 seconds for 08E
Query returned 38588 distances in 17.94 seconds for 08F
Query returned 23684 distances in 16.38 seconds for 08G
Query returned 100507 distances in 22.47 seconds for 10E
Query returned 125217 distances in 21.37 seconds for CLR
Query returned 172787 distances in 24.51 seconds for ERK
Query returned 166028 distances in 27.64 seconds for FRA
Query returned 47842 distances in 18.53 seconds for HAY
Query returned 5541 distances in 14.62 seconds for HGW
Query returned 163680 distances in 27.14 seconds for LRD
Query returned 104650 distances in 22.58 seconds for PCR
Query returned 20418 distances in 15.84 seconds for VCI
Query returned 44232 distances in 17.23 seconds for WWA
Query returned 49442 distances in 17.69 seconds for YKR
